In [3]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [4]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/MIP_1_Facility_Location_Allocation/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-18 11:36:37.518166


## Step 1 - Generate Mathematical Formulation 

In [5]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [6]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [7]:
Markdown(response_1_text)

This problem can be formulated as a Mixed-Integer Linear Programming (MILP) problem, specifically a variant of the Uncapacitated Facility Location Problem (UFLP) or Capacitated Facility Location Problem (CFLP), depending on whether the facilities have a maximum service volume or not. Here's the mathematical optimization model:

**Parameters:**

- $I$: Set of potential facility locations, indexed by $i$.
- $J$: Set of customers, indexed by $j$.
- $f_i$: Fixed annual activation cost of opening facility $i$.
- $c_{ij}$: Transportation cost of servicing customer $j$ from facility $i$.
- $d_j$: Demand of customer $j$.
- $s_i$: Maximum service volume of facility $i$ (if applicable).

**Decision Variables:**

- $x_{ij} \in \{0, 1\}$: Binary variable that is equal to 1 if customer $j$ is serviced by facility $i$, and 0 otherwise.
- $y_i \in \{0, 1\}$: Binary variable that is equal to 1 if facility $i$ is opened, and 0 otherwise.

**Objective Function:**

Minimize the total cost, which is the sum of the fixed activation costs and the transportation costs:

$$ \text{min} \quad \sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} $$

**Constraints:**

1. Each customer's demand must be met by exactly one facility:
   $$ \sum_{i \in I} x_{ij} = 1 \quad \forall j \in J $$

2. Customers can only be serviced by an opened facility:
   $$ x_{ij} \leq y_i \quad \forall i \in I, j \in J $$

3. If the facilities have a maximum service volume, then each facility's service volume must not exceed its maximum limit:
   $$ \sum_{j \in J} d_j x_{ij} \leq s_i y_i \quad \forall i \in I $$

This optimization model can be solved using a MILP solver to find the optimal locations for opening facilities and assigning customers to those facilities, minimizing the overall costs.

In [8]:
print(response_1_text)

This problem can be formulated as a Mixed-Integer Linear Programming (MILP) problem, specifically a variant of the Uncapacitated Facility Location Problem (UFLP) or Capacitated Facility Location Problem (CFLP), depending on whether the facilities have a maximum service volume or not. Here's the mathematical optimization model:

**Parameters:**

- $I$: Set of potential facility locations, indexed by $i$.
- $J$: Set of customers, indexed by $j$.
- $f_i$: Fixed annual activation cost of opening facility $i$.
- $c_{ij}$: Transportation cost of servicing customer $j$ from facility $i$.
- $d_j$: Demand of customer $j$.
- $s_i$: Maximum service volume of facility $i$ (if applicable).

**Decision Variables:**

- $x_{ij} \in \{0, 1\}$: Binary variable that is equal to 1 if customer $j$ is serviced by facility $i$, and 0 otherwise.
- $y_i \in \{0, 1\}$: Binary variable that is equal to 1 if facility $i$ is opened, and 0 otherwise.

**Objective Function:**

Minimize the total cost, which is the s

## Step 2 - Generate the Pyomo Code

In [9]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [10]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [11]:
Markdown(response_2_text)

Here is a Python code using Pyomo to solve the Capacitated Facility Location Problem (CFLP) as described. I will use sample data for parameters such as $I$, $J$, $f_i$, $c_{ij}$, $d_j$, and $s_i$.

```python
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Sample data
I = [1, 2, 3]  # potential facility locations
J = [1, 2, 3]  # customers
f_i = {1: 500, 2: 300, 3: 700}  # fixed activation costs
c_ij = {(1, 1): 10, (1, 2): 20, (1, 3): 30,
        (2, 1): 15, (2, 2): 25, (2, 3): 35,
        (3, 1): 20, (3, 2): 30, (3, 3): 40}  # transportation costs
d_j = {1: 50, 2: 60, 3: 70}  # demands
s_i = {1: 150, 2: 180, 3: 220}  # maximum service volumes

# Create Pyomo model
model = pyo.ConcreteModel("Capacitated Facility Location Problem")

# Define sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Define parameters
model.fixed_activation_costs = pyo.Param(model.I, initialize=f_i)
model.transportation_costs = pyo.Param(model.I, model.J, initialize=c_ij)
model.demands = pyo.Param(model.J, initialize=d_j)
model.max_service_volumes = pyo.Param(model.I, initialize=s_i)

# Define decision variables
model.x_ij = pyo.Var(model.I, model.J, domain=pyo.Binary)
model.y_i = pyo.Var(model.I, domain=pyo.Binary)

# Define the objective function
def objective_rule(model):
    return pyo.summation(model.fixed_activation_costs, model.y_i) + \
           pyo.summation(model.transportation_costs, model.x_ij)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints
def demand_constraints_rule(model, j):
    return pyo.summation(model.x_ij, model.I) == 1
model.demand_constraints = pyo.Constraint(model.J, rule=demand_constraints_rule)

def facility_opening_constraints_rule(model, i, j):
    return model.x_ij[i, j] <= model.y_i[i]
model.facility_opening_constraints = pyo.Constraint(model.I, model.J,
                                                   rule=facility_opening_constraints_rule)

def capacity_constraints_rule(model, i):
    return pyo.summation(model.demands, model.x_ij[i]) <= model.max_service_volumes[i] * model.y_i[i]
model.capacity_constraints = pyo.Constraint(model.I, rule=capacity_constraints_rule)

# Solve the model
optimizer = SolverFactory('glpk')
optimizer.solve(model)

# Print the solution
print("x_ij values")
for i in model.I:
    for j in model.J:
        print(f"x_{i}_{j}: {model.x_ij[i, j].value}")

print("\ny_i values")
for i in model.I:
    print(f"y_{i}: {model.y_i[i].value}")

print("\nObjective value: ", model.objective.expr().value)
```

This code uses Pyomo to model and solve the CFLP using the GLPK solver. You can replace the sample data with your own data and use a different MILP solver if needed.

In [12]:
print(response_2_text)

Here is a Python code using Pyomo to solve the Capacitated Facility Location Problem (CFLP) as described. I will use sample data for parameters such as $I$, $J$, $f_i$, $c_{ij}$, $d_j$, and $s_i$.

```python
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Sample data
I = [1, 2, 3]  # potential facility locations
J = [1, 2, 3]  # customers
f_i = {1: 500, 2: 300, 3: 700}  # fixed activation costs
c_ij = {(1, 1): 10, (1, 2): 20, (1, 3): 30,
        (2, 1): 15, (2, 2): 25, (2, 3): 35,
        (3, 1): 20, (3, 2): 30, (3, 3): 40}  # transportation costs
d_j = {1: 50, 2: 60, 3: 70}  # demands
s_i = {1: 150, 2: 180, 3: 220}  # maximum service volumes

# Create Pyomo model
model = pyo.ConcreteModel("Capacitated Facility Location Problem")

# Define sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Define parameters
model.fixed_activation_costs = pyo.Param(model.I, initialize=f_i)
model.transportation_costs = pyo.Param(model.I, model.J, initialize=c_ij)
m

### Code Executability

In [24]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Sample data
I = [1, 2, 3]  # potential facility locations
J = [1, 2, 3]  # customers
f_i = {1: 500, 2: 300, 3: 700}  # fixed activation costs
c_ij = {(1, 1): 10, (1, 2): 20, (1, 3): 30,
        (2, 1): 15, (2, 2): 25, (2, 3): 35,
        (3, 1): 20, (3, 2): 30, (3, 3): 40}  # transportation costs
d_j = {1: 50, 2: 60, 3: 70}  # demands
s_i = {1: 150, 2: 180, 3: 220}  # maximum service volumes

# Create Pyomo model
model = pyo.ConcreteModel("Capacitated Facility Location Problem")

# Define sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Define parameters
f_i = pyo.Param(model.I, initialize=f_i)
model.transportation_costs = pyo.Param(model.I, model.J, initialize=c_ij)
model.demands = pyo.Param(model.J, initialize=d_j)
model.max_service_volumes = pyo.Param(model.I, initialize=s_i)

# Define decision variables
model.x_ij = pyo.Var(model.I, model.J, domain=pyo.Binary)
model.y_i = pyo.Var(model.I, domain=pyo.Binary)

# Define the objective function
def objective_rule(model):
    return pyo.summation(f_i, model.y_i) + \
           pyo.summation(model.transportation_costs, model.x_ij)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints
def demand_constraints_rule(model, j):
    return pyo.summation(model.x_ij, model.I) == 1
model.demand_constraints = pyo.Constraint(model.J, rule=demand_constraints_rule)

def facility_opening_constraints_rule(model, i, j):
    return model.x_ij[i, j] <= model.y_i[i]
model.facility_opening_constraints = pyo.Constraint(model.I, model.J,
                                                   rule=facility_opening_constraints_rule)

def capacity_constraints_rule(model, i):
    return pyo.summation(model.demands, model.x_ij[i]) <= model.max_service_volumes[i] * model.y_i[i]
model.capacity_constraints = pyo.Constraint(model.I, rule=capacity_constraints_rule)

# Solve the model
optimizer = SolverFactory('glpk')
optimizer.solve(model)

# Print the solution
print("x_ij values")
for i in model.I:
    for j in model.J:
        print(f"x_{i}_{j}: {model.x_ij[i, j].value}")

print("\ny_i values")
for i in model.I:
    print(f"y_{i}: {model.y_i[i].value}")

print("\nObjective value: ", model.objective.expr().value)

2024-05-18 11:52:50,467 ERROR pyomo.core: Rule failed for Param 'demands' with index 4:
KeyError: "Index '4' is not valid for indexed component 'demands'"
2024-05-18 11:52:50,468 ERROR pyomo.core: Constructing component 'demands' from data=None failed:
KeyError: "Index '4' is not valid for indexed component 'demands'"


KeyError: "Index '4' is not valid for indexed component 'demands'"

### Solution Correctness

In [9]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Sample data
f_i = {1: 1000, 2: 1000, 3: 1000}  # fixed activation costs
c_ij = {
    (1, 1): 4,  (1, 2): 6,  (1, 3): 9,
    (2, 1): 5,  (2, 2): 4,  (2, 3): 7,
    (3, 1): 6,  (3, 2): 3,  (3, 3): 4,
    (4, 1): 8,  (4, 2): 5,  (4, 3): 3,
    (5, 1): 10, (5, 2): 8,  (5, 3): 4,
        }  # transportation costs
d_j = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}  # demands
s_i = {1: 500, 2: 500, 3: 500}  # maximum service volumes

I = list(s_i.keys())  # potential facility locations
J = list(d_j.keys())

# Create Pyomo model
model = pyo.ConcreteModel("Capacitated Facility Location Problem")

# Define sets
model.I = pyo.Set(initialize=J)
model.J = pyo.Set(initialize=J)

# Define parameters
# model.fixed_activation_costs = pyo.Param(I, initialize=f_i)
# model.transportation_costs = pyo.Param(I, J, initialize=c_ij)
# model.demands = pyo.Param(J, initialize=d_j)
# model.max_service_volumes = pyo.Param(I, initialize=s_i)

# Define decision variables
model.x_ij = pyo.Var(model.I, model.J, domain=pyo.Binary)
model.y_i = pyo.Var(model.I, domain=pyo.Binary)

# Define the objective function
def objective_rule(model):
    return sum(f_i[i] * model.y_i[i] for i in model.I)\
    + sum(sum(c_ij[j, i] * model.x_ij[i, j] for i in model.I) for j in model.J)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints
def demand_constraints_rule(model, j):
    return sum(model.x_ij[i, j] for i in model.I) == 1
d_j = pyo.Constraint(model.J, rule=demand_constraints_rule)

def facility_opening_constraints_rule(model, i, j):
    return model.x_ij[i, j] <= model.y_i[i]
model.facility_opening_constraints = pyo.Constraint(model.I, model.J,
                                                   rule=facility_opening_constraints_rule)

def capacity_constraints_rule(model, i):
    return sum(d_j * model.x_ij[i, j] for j in model.J) <= s_i[i] * model.y_i[i]
model.capacity_constraints = pyo.Constraint(model.I, rule=capacity_constraints_rule)

# Solve the model
optimizer = SolverFactory('glpk')
optimizer.solve(model)

# Print the solution
print("x_ij values")
for i in model.I:
    for j in model.J:
        print(f"x_{i}_{j}: {model.x_ij[i, j].value}")

print("\ny_i values")
for i in model.I:
    print(f"y_{i}: {model.y_i[i].value}")

print("\nObjective value: ", model.objective.expr())

ERROR: Rule failed when generating expression for Objective objective with
    index None: KeyError: 4
ERROR: Constructing component 'objective' from data=None failed: KeyError: 4


KeyError: 4

In [5]:
model.pprint()

6 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    fixed_activation_costs_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    transportation_costs_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                                                    : Size : Members
        None :     2 : transportation_costs_index_0*transportation_costs_index_1 :   15 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5)}
    transportation_costs_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    A